In [ ]:
import pandas as pd
import numpy as np

from gee_scripts.model import create_hybrid_model, create_simplified_model
from gee_scripts.parameters import explain_vars, response_var

In [ ]:
df = pd.read_csv("data/9_clean_training_data/clean_training_data.csv")

# Dummy data with 1000 samples and 30 features
X = df[explain_vars].values
y = df[response_var].values

In [ ]:
# Create and compile the model
train_model = create_simplified_model(len(explain_vars))
train_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
train_model.fit(X, y, epochs=500, batch_size=32)


In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

# Function to create model, required for KerasRegressor
def create_model(layers=[10], activation='relu'):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i == 0:
            model.add(Dense(nodes, input_dim=28, activation=activation))
        else:
            model.add(Dense(nodes, activation=activation))
    model.add(Dense(1))  # Output layer
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

# Create the model
model = KerasRegressor(build_fn=create_model, verbose=0)

# Define the grid search parameters
param_grid = {
    'batch_size': [16, 32],
    'epochs': [10, 50, 100, 200],
    'layers': [[20], [40, 20], [45, 30, 15]],
    'activation': ['sigmoid', 'relu']
}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X, y)

# Summarize results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")


In [ ]:
# use the result of the grid search to create the model
estimator = create_model(
    layers=[45, 30, 15],
    activation='sigmoid'
)

# Train the model
estimator.fit(X, y, epochs=10, batch_size=32)

y_pred = estimator.predict([X_test_lstm, X_test_dense])

In [ ]:
"""
/tmp/ipykernel_163580/970427751.py:19: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model, verbose=0)
2023-11-02 09:23:26.998294: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Best: -215.53753407796225 using {'activation': 'sigmoid', 'batch_size': 32, 'epochs': 10, 'layers': [45, 30, 15]}"""